In [2]:
!pip install transformers==4.28.0 sentencepiece sacremoses datasets 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 40.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=800796a2a63a83c4ca992d35d269c3f64b73a4d44edba72c7f97f369b4f4b6f4
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("HF_KEY")
login(secret_value)
secret_value1 = user_secrets.get_secret("WANDB_KEY")
wandb.login(key=secret_value1)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
from datasets import load_dataset
training_data = load_dataset("ethansimrm/wmt_16_19_22_biomed_train", split = "train", use_auth_token=True) 

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/ethansimrm--wmt_16_19_22_biomed_train-3dd4d6328c3dbab1/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


In [6]:
#len(training_data) #741300 sentence pairs prior to pre-processing
#training_data['text'][1:10] #Adopting the SOURCE \t TARGET \n format is convenient.

741300

In [5]:
from datasets import Dataset
source = []
target = []
for example in training_data['text']:
    example = example.strip()
    sentences = example.split("\t")
    source.append(sentences[0])
    target.append(sentences[1])
ready_data = Dataset.from_dict({"en":source, "fr":target})
#This dataset can now be used for training or validation per early experiments

In [6]:
#Seems alright.
ready_data[0]

{'en': 'Global Health: Where Do Physiotherapy and Rehabilitation Research Fit?',
 'fr': 'La place des cheveux et des poils dans les rituels et le sacré.'}

In [7]:
import pandas as pd
#Let's start preprocessing according to Wu et al.(2022).
#First, remove duplicate sentences. 
#We can't do this in datasets, so we must convert it to a pandas DataFrame.
tempDataset = pd.DataFrame(ready_data)

In [43]:
#tempDataset[tempDataset.duplicated()] 
#Wow, there are a lot of duplicate rows, 39540 in total.

,en,fr
664,Editorial.,Éditorial.
1291,Editorial.,Éditorial.
2163,Drugs news.,Info-médicaments.
2750,Editorial.,Éditorial.
3727,Misophonia and contemporary psychiatry.,Misophonie et psychiatrie contemporaine.
...,...,...
738932,"Unfortunately, program decisions for training ...","Malheureusement, la décision de la direction d..."
738933,"This series of three articles, intended for th...","Cette série de trois articles, destinée aux re..."
738934,With these 14 recommendations on how to get a ...,La mise en œuvre de ces 14 recommandations sur...
738935,They are offered to minimize the chances of ac...,Elles sont proposées pour éviter la révision d...


In [8]:
tempDataset_dedup = tempDataset.drop_duplicates()

In [28]:
#Declaring helper functions separately
import langid, string
from tqdm import tqdm
tqdm.pandas() #So I can see progress
count = lambda l1,l2: sum([1 for x in l1 if x in l2])

In [29]:
#Having verified that we can convert this back into a Dataset object, we'll just 
#do everything else in a pandas DataFrame. There are five per-sentence checks
#we can do.

def retain_sentence(row): #This defines a good sentence we wish to retain 
    for sentence in (row["en"], row["fr"]):
        #Exclude sentences with mismatched () or '' or ""
        if ((sentence.count("(") != sentence.count(")")) or 
            (sentence.count("'") % 2 != 0) or
            (sentence.count('"') % 2 != 0)):
            return False
        sent_length = len(sentence)
        #Exclude sentences with punctuation percentage > 0.4
        if (count(sentence,set(string.punctuation)) > 0.4 * sent_length):
            return False
        #Exclude sentences with > 150 words
        num_words = len(sentence.split(" "))
        if (num_words > 150):
            return False
        #Exclude sentences with char-to-word ratio > 12 or < 1.5
        c2w_ratio = sent_length / num_words
        if ((c2w_ratio > 12) or (c2w_ratio < 1.5)):
            return False
    #Exclude sentences which don't match the correct language (costly)
    if ((langid.classify(row['en'])[0] != "en") or 
        (langid.classify(row['fr'])[0] != "fr")):
        return False
    #There is one more step which involves checking sentence alignment; we will
    #do that on the command line because it's a bit more involved.
    return True

In [30]:
tempDataset_nearlyAligned = tempDataset_dedup.progress_apply(retain_sentence, axis=1)

100%|██████████| 701760/701760 [26:05<00:00, 448.38it/s] 


In [33]:
filtered_set = tempDataset_dedup[tempDataset_nearlyAligned]

In [45]:
filtered_set = filtered_set.reset_index()

In [48]:
f = open("filtered.txt", "w", encoding = "utf-8")
source = filtered_set["en"]
target = filtered_set["fr"]
for i in range(len(filtered_set)):
    f.write(source[i] + "\t" + target[i] + "\n")
f.close()